# Protein Folding in 2D 
Proteins are long strands of amino acids that control many important processes in the human body. It is known that proteins are stored 'folded' inside the cells of the body, and that the specific folding significantly influences their functioning. ‘Misfolded’ proteins can play a role in cancer, Alzheimer's disease and cystic fibrosis.

Due to the complexity of the protein folding problem, simplified models such as Dill's hydrophobic-polar (HP) model have become one of the major tools for studying protein structure. The HP model is based on the observation that the hydrophobic force is the main force deter- mining the unique native conformation (and hence the functional state) of small globular proteins.

In a protein, hydrophobic amino acids (H) like to lie 'adjecntly', polar amino acids (P) do not have that preference. When two hydrophobic amino acids lie next to each other, an 'H- bond' is formed due to the attractive forces between the two. And the more bonds, the more stable the protein. The HP-model, containing of just two types of amino acids, arranges these in a grid or a lattice.

![Example Screenshot](8_piece_protein.png)


In [1]:
import numpy as np 
import pandas as pd

## Basic Algorithm for Protein Folding
This thread covers the development of an algorithm to simulate and analyze protein folding using a simplified model known as the HP model. The HP model involves two types of amino acids: hydrophobic (H) and polar (P). The goal is to arrange these amino acids on a 2D grid or lattice to create stable protein configurations.

The key steps of the algorithm include:

1. **Initializing Acids**: A function called `random_division` is used to divide a specified number of amino acids into hydrophobic and polar types while ensuring at least one of each type. 
- **Initializing Grid**: The `initialize grid` function begins by initializing a 2D grid or lattice with a specified size. Hydrophobic and polar amino acids are randomly placed on the grid.
- **Calculate Energy**: The function `calculate_hbonds` calculates the initial energy of a given grid by counting the number of adjacent hydrophobic amino acids (H-bonds).

- **Fold Proteins**: The function `reshape_grid` attempts to reshape a given grid to improve stability. It iterates through various configurations to optimize the placement of amino acids and maximizes the energy function.

- **Creating Samples**: The `generate_random_samples` function generates a given number of random samples for various amino acid lengths. It uses the random division to determine the amino acid types and grid size and then initializes grids accordingly.

- **Generating and storing Results**: The code generates random samples for different amino acid lengths (5, 10, 15, 20, 25, 30, 35, and 40) and calculates various metrics for each sample. It then prints the sample information, grid configuration, and metrics.


### Step 1 - Initializing Acids

This function `random_division` takes the total number of amino acids as input and randomly divides them into hydrophobic and polar amino acids, before creating a random 1D amino acid string. It ensures that there's at least one hydrophobic and one polar amino acid. You can adjust the total_amino_acids variable to the desired number of amino acids you want to divide.

In [74]:
total_amino_acids = 10  # Change this to a desired number, 10 is used for illustration purposes

def random_division(total_amino_acids):
    num_hydrophobic = np.random.randint(1, total_amino_acids)  # Ensure at least one H and P
    num_polar = total_amino_acids - num_hydrophobic
    return num_hydrophobic, num_polar

num_hydrophobic, num_polar = random_division(total_amino_acids)
print("Number of Hydro acids:", num_hydrophobic)
print("Number of Polar acids:", num_polar)

# randomly shuffle the amino acids
amino_acids = ['H'] * num_hydrophobic + ['P'] * num_polar
np.random.shuffle(amino_acids)

print("Random Amino Acid String:", "".join(amino_acids))

Number of Hydro acids: 9
Number of Polar acids: 1
Random Amino Acid String: HHHHPHHHHH


### Step 2 - Initializing Grid
The `initialize_grid` function generates a grid to place amino acids, with specified dimensions, and populates it with H and P amino acids, while ensuring they do not overlap and tracks their placement order. This allows for simulating the random arrangement of amino acids on a grid for various applications.

In [71]:
amino_acids_copy = amino_acids.copy()

def initialize_grid(amino_acids, num_hydrophobic, num_polar):
    grid_size = num_hydrophobic + num_polar
    total_amino_acids = len(amino_acids)
    grid = np.full((grid_size, grid_size), fill_value='', dtype=object)  
    amino_acid_order = []  # List to track the order of amino acids placed
    
    # Place the first amino acid randomly on the grid
    first_amino_row = np.random.randint(grid_size)
    first_amino_col = np.random.randint(grid_size)
    amino_acid_type = amino_acids_copy.pop(0)
    grid[first_amino_row, first_amino_col] = amino_acid_type
    
    # Keep track of order
    amino_acid_order.append((amino_acid_type, (first_amino_row, first_amino_col)))
    total_amino_acids -= 1
    
    # Place the remaining amino acids next to the last placed amino acid
    while total_amino_acids > 0:
        last_amino_row, last_amino_col = amino_acid_order[-1][1]
        neighbors = [
            (last_amino_row - 1, last_amino_col),
            (last_amino_row + 1, last_amino_col),
            (last_amino_row, last_amino_col - 1),
            (last_amino_row, last_amino_col + 1)
        ]
        
        valid_neighbors = [(row, col) for row, col in neighbors if 0 <= row < grid_size and 0 <= col < grid_size and grid[row, col] == '']
        
        if valid_neighbors and amino_acids_copy:  # Check if there are valid neighbors and remaining amino acids
            # Shuffle the list of valid neighbors and choose one randomly
            np.random.shuffle(valid_neighbors)
            chosen_row, chosen_col = valid_neighbors[0]
            amino_acid_type = amino_acids_copy.pop(0)
            grid[chosen_row, chosen_col] = amino_acid_type
            amino_acid_order.append((amino_acid_type, (chosen_row, chosen_col)))
            total_amino_acids -= 1
        else:
            break  # No valid neighbors or no remaining amino acids, exit the loop
    
    return grid, amino_acid_order

# Example usage
initial_grid, amino_acid_order = initialize_grid( amino_acids_copy, num_hydrophobic, num_polar)


In [72]:
print("Grid size:", grid_size, 'x', grid_size)
print(initial_grid)


Grid size: 10 x 10
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' 'H' '' '' '' '']
 ['' '' '' '' '' 'P' 'H' 'P' '' '']
 ['' '' '' '' '' '' 'H' 'P' 'P' '']
 ['' '' '' '' '' '' 'P' 'H' 'H' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']]


In [66]:
print("Amino Acid Order:", amino_acid_order)

Amino Acid Order: [('P', (9, 3)), ('P', (9, 2)), ('P', (9, 1)), ('H', (8, 1)), ('H', (8, 2)), ('P', (8, 3)), ('H', (7, 3)), ('H', (7, 2)), ('P', (7, 1)), ('H', (6, 1))]


### Step 3a - Calculate H-bonds
The `find_H_combinations_grid` function is used to~ identify and collect sets of coordinates representing adjacent 'H' amino acids in a grid. It iterates through the entire grid, checking each position for the presence of 'H' amino acids. If an 'H' amino acid is found, it examines neighboring positions (up, down, left, right) to identify adjacent 'H' amino acids. For each pair of adjacent 'H' amino acids, it creates a frozenset containing their coordinates (ensuring that the order of coordinates doesn't matter) and adds this frozenset to a set. This set stores all unique pairs of adjacent 'H' amino acids found in the grid. The function returns this set of adjacent 'H' amino acid pairs.

In [67]:
def find_H_pairs_grid(grid):
    adjacent_hydrophobic_amino_acids = set()  # Use a set to automatically remove duplicates

    # Iterate through the grid to find adjacent 'H' amino acids
    for row in range(grid.shape[0]):
        for col in range(grid.shape[1]):
            current_acid = grid[row, col]

            # Check if the current amino acid is 'H'
            if current_acid == 'H':
                # Check the neighboring positions (up, down, left, right) relative to the current position
                neighbors = [
                    (row - 1, col),
                    (row + 1, col),
                    (row, col - 1),
                    (row, col + 1)
                ]

                for neighbor_row, neighbor_col in neighbors:
                    # Check if the neighbor is within the grid bounds
                    if 0 <= neighbor_row < grid.shape[0] and 0 <= neighbor_col < grid.shape[1]:
                        neighbor_acid = grid[neighbor_row, neighbor_col]

                        # Check if the neighbor is also 'H'
                        if neighbor_acid == 'H':
                            # Use frozenset to ensure that the order of coordinates doesn't matter
                            amino_acid_pair = frozenset({(row, col), (neighbor_row, neighbor_col)})
                            adjacent_hydrophobic_amino_acids.add(amino_acid_pair)
                            
    return adjacent_hydrophobic_amino_acids

find_H_pairs_grid(initial_grid)

{frozenset({(7, 2), (7, 3)}),
 frozenset({(7, 2), (8, 2)}),
 frozenset({(8, 1), (8, 2)})}

### Step 3b - Calculate H-bonds
The `find_H_combinations_order` function examines the order of amino acids and identifies adjacent 'H' amino acids. It does this by iterating through the amino acid order, checking pairs of consecutive amino acids for 'H' type, and recording these pairs as frozensets in a set to remove duplicates. This function helps identify adjacent 'H' amino acids in the sequence order, which is useful for analyzing the arrangement of amino acids.

In [68]:
def find_H_pairs_order(amino_acid_order):
    adjacent_hydrophobic_amino_acids = set()  # Use a set to automatically remove duplicates

    # Iterate through the amino acid order to find adjacent 'H' amino acids
    for i in range(len(amino_acid_order) - 1):
        current_acid, current_position = amino_acid_order[i]
        next_acid, next_position = amino_acid_order[i + 1]

        # Check if both current and next amino acids are 'H'
        if current_acid == 'H' and next_acid == 'H':
            # Use frozenset to ensure that the order of positions doesn't matter
            amino_acid_pair = frozenset({current_position, next_position})
            adjacent_hydrophobic_amino_acids.add(amino_acid_pair)

    return adjacent_hydrophobic_amino_acids

find_H_pairs_order(amino_acid_order)

{frozenset({(8, 1), (8, 2)}), frozenset({(7, 2), (7, 3)})}

### Step 3c - Calculate H-bonds
The provided code calculates the H-combinations in two different ways and then compares them. It calculates the H-pairs in the grid using the `find_H_combinations_grid` function and the H-pairs in the amino acid order using the `find_H_combination_order` function. Then, it subtracts the H-combinations found in the amino acid order from those found in the grid. This comparison helps identify the **H-bonds** that are formed between adjacent 'H' amino acids in the grid but not in the given amino acid sequence order.

In [69]:
def find_H_bonds(grid, amino_acid_order):
    grid_h_pairs = find_H_pairs_grid(grid)
    order_h_pairs = find_H_pairs_order(amino_acid_order)
    return grid_h_pairs - order_h_pairs

H_bonds = find_H_bonds(initial_grid, amino_acid_order)

In [70]:
print('Number of H-bonds:', len(H_bonds))

for bond in H_bonds:
    coordinates = [coord for coord in bond]
    print(coordinates)

Number of H-bonds: 1
[(8, 2), (7, 2)]


### Step 3 - Fold Proteins
Now we implement an Ant Colony Optimization (ACO) algorithm to optimize the folding of amino acids on a grid to maximize H-bonds. It initializes the grid with random amino acid placements, and ants iteratively move amino acids on the grid while considering pheromone levels and a heuristic function based on the H-bond score. The pheromone levels are updated based on the best placement found by ants. Finally, the best placement and H-bonds score are printed.

In [33]:
initial_grid

array([['', '', '', '', '', '', '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', ''],
       ['', '', '', '', '', '', '', '', '', ''],
       ['', '', '', 'P', 'H', '', '', '', '', ''],
       ['', '', '', '', 'P', 'H', '', 'P', 'P', ''],
       ['', '', '', '', '', 'P', 'P', 'P', 'P', ''],
       ['', '', '', '', '', '', '', '', '', '']], dtype=object)

In [34]:
print(len(amino_acids))
def random_search(grid, amino_acids, num_iterations):
    best_grid = np.copy(grid)
    best_hbonds = len(find_H_bonds(initial_grid, amino_acid_order))

    for _ in range(num_iterations):
        # Randomly select two positions
        pos1 = np.random.randint(0, len(amino_acids))
        pos2 = np.random.randint(0, len(amino_acids))

        # Swap the amino acids at those positions
        amino_acids[pos1], amino_acids[pos2] = amino_acids[pos2], amino_acids[pos1]

        # Update the grid
        np.random.shuffle(amino_acids)
        new_grid, _ = initialize_grid(grid.shape[0], amino_acids)

        # Calculate H-bonds for the new configuration
        new_hbonds = calculate_hbonds(new_grid)

        # Accept the new configuration if it's better
        if new_hbonds > best_hbonds:
            best_hbonds = new_hbonds
            best_grid = new_grid
        else:
            # Revert the swap
            amino_acids[pos1], amino_acids[pos2] = amino_acids[pos2], amino_acids[pos1]

    return best_grid, best_hbonds

# Example usage
random_search(initial_grid, amino_acids, 100)

# print("Initial Grid:")
# print(initial_grid)
# print("Initial H-bonds:", calculate_hbonds(initial_grid))
# print("\nOptimized Grid:")
# print(optimized_grid)
# print("Optimized H-bonds:", optimized_hbonds)

print(len(amino_acids))

10


NameError: name 'calculate_hbonds' is not defined

In [35]:
len(amino_acids)

10

### Step 4 - Creating Samples
The code generates N random protein grid configurations with a specified number of hydrophobic and polar amino acids using the `generate_random_samples` function. 

In [73]:
def generate_random_samples(N, total_amino_acids, grid_size):
    random_samples = []

    for _ in range(N):
        num_hydrophobic, num_polar, amino_acids = random_division(total_amino_acids)
        initial_grid, amino_acid_order = initialize_grid(amino_acids, num_hydrophobic, num_polar)
        hbonds = find_H_bonds(initial_grid, amino_acid_order)
        
        random_samples.append((initial_grid, amino_acid_order, hbonds))

    return random_samples

# Example usage
N = 5  # Number of random samples
total_amino_acids = 10  # Total number of amino acids
grid_size = int(np.sqrt(total_amino_acids))  # Size of the grid

random_samples = generate_random_samples(N, total_amino_acids, grid_size)

for i, (initial_grid, amino_acid_order, hbonds) in enumerate(random_samples):
    print(f"Sample {i + 1}:")
    print("Grid:")
    print(initial_grid)
    print("Amino Acid Order:")
    print(amino_acid_order)
    print("H-bonds:", len(hbonds))
    print()

ValueError: not enough values to unpack (expected 3, got 2)

In [40]:
def generate_random_samples(N, grid_size):
    random_samples = []

    for _ in range(N):
        num_hydrophobic, num_polar = random_division(grid_size**2)
        amino_acids = ['H'] * num_hydrophobic + ['P'] * num_polar
        np.random.shuffle(amino_acids)
        amino_acids_copy = amino_acids.copy()
        
        initial_grid, amino_acid_order = initialize_grid(grid_size, amino_acids_copy)
        hbonds = find_H_bonds(initial_grid, amino_acid_order)
        
        random_samples.append((initial_grid, amino_acid_order, hbonds))

    return random_samples

# Example usage
N = 5  # Number of random samples
grid_size = 10  # Size of the grid
random_samples = generate_random_samples(N, grid_size)

for i, (initial_grid, amino_acid_order, hbonds) in enumerate(random_samples):
    print(f"Sample {i + 1}:")
    print("Grid:")
    print(initial_grid)
    print("Amino Acid Order:")
    print(amino_acid_order)
    print("H-bonds:", hbonds)
    print()

Sample 1:
Grid:
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' 'P' 'P']
 ['' '' '' '' '' '' '' '' 'P' '']
 ['' '' '' '' '' '' '' 'P' 'P' '']
 ['' '' '' '' '' '' '' 'H' 'H' 'P']
 ['' '' '' '' '' '' '' '' '' 'P']
 ['' '' '' '' '' '' '' '' '' 'P']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']]
Amino Acid Order:
[('P', (7, 9)), ('P', (6, 9)), ('P', (5, 9)), ('H', (5, 8)), ('H', (5, 7)), ('P', (4, 7)), ('P', (4, 8)), ('P', (3, 8)), ('P', (2, 8)), ('P', (2, 9))]
H-bonds: set()

Sample 2:
Grid:
[['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' 'P' 'P']
 ['' '' '' '' '' '' 'P' 'P' '' 'P']
 ['' '' '' '' '' '' '' 'P' 'H' 'P']
 ['' '' '' '' '' '' '' '' 'H' 'P']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '']]
Amino Acid Order:
[('P', (4, 6)), ('P', (4, 7)), ('P', (5, 7)), ('H', (5, 8)), ('H', (6, 8)), ('P', (

In [165]:
amino_acid_lengths = [5, 10, 15, 20, 25, 30, 35, 40]

data = {
    "Amino Acid Length": [],
    "Num Hydrophobic": [],
    "Num Polar": [],
    "Initial Score": [],
    "Reshaped Score": [],
    "HC Improvement": []
}

for length in amino_acid_lengths:
    for _ in range(num_samples):
        total_amino_acids = length
        random_samples = generate_random_samples(1, total_amino_acids)[0]
        
        num_hydrophobic = np.sum(random_samples == 'H')
        num_polar = np.sum(random_samples == 'P')
        initial_score = calculate_score(random_samples)
        
        reshaped_grid, reshaped_score = reshape_grid(random_samples, iterations=10000)  # Adjust iterations as needed
        hc_improvement = initial_score - reshaped_score
        
        data["Amino Acid Length"].append(length)
        data["Num Hydrophobic"].append(num_hydrophobic)
        data["Num Polar"].append(num_polar)
        data["Initial Score"].append(initial_score)
        data["Reshaped Score"].append(reshaped_score)
        data["HC Improvement"].append(hc_improvement)

df = pd.DataFrame(data)
df.head()

,Amino Acid Length,Num Hydrophobic,Num Polar,Initial Score,Reshaped Score,HC Improvement
0,5,3,2,2,2,0
1,5,2,3,1,1,0
2,5,2,3,1,1,0
3,5,1,4,0,0,0
4,5,1,4,0,0,0


In [153]:
df['HC Improvement'].max()

0